# 📉 Estrategia Miércoles: Mínima Varianza Diversificada

## 1. Justificación Táctica
Analizando el rendimiento de la cartera del martes (Sharpe 0.596), identificamos una **concentración excesiva de riesgo** en un único activo (que representaba >30% de la cartera). Esto disparó la **Kurtosis (19.4)**, indicando vulnerabilidad ante eventos específicos de ese activo.

Para la entrega del miércoles, mantenemos la filosofía de Mínima Varianza Global (usando la covarianza muestral estándar enseñada en el curso) pero introducimos una **Restricción de Concentración**.

## 2. Formulación Matemática

$$
\begin{aligned}
& \underset{w}{\text{minimizar}} & & \sigma^2 = w^T \Sigma w \\
& \text{sujeto a:} & & \sum w_i = 0.90 \quad (\text{Liquidez Táctica}) \\
& & & w_i \geq 0 \\
& & & w_i \leq 0.22 \quad (\text{Límite de Concentración})
\end{aligned}
$$

### Explicación de las Restricciones:
1.  **Objetivo:** Minimizar la volatilidad usando la Matriz de Covarianza Muestral ($\Sigma$).
2.  **$\sum w_i = 0.90$:** Mantenemos un 10% en el Activo Libre de Riesgo para mejorar el perfil riesgo-retorno (CML).
3.  **$w_i \leq 0.22$:** Imponemos un "techo" duro del 22% por activo.
    * *Efecto:* Esto fuerza matemáticamente la diversificación entre los 5 activos seleccionados. Al obligar a repartir los pesos de forma más equitativa, suavizamos las colas de la distribución de retornos y reducimos la Kurtosis sin necesidad de cambiar el modelo subyacente.

El motivo por el que he puesto en límite en 22% y no en 25% es porque de la segunda forma el asset 45 tenía el doble de peso que el asset 23, de esta forma está mejor repartido.

In [1]:
import pandas as pd
import numpy as np
import cvxpy as cp

def optimizar_miercoles_limite_22():
    print("--- ESTRATEGIA MIÉRCOLES: DIVERSIFICACIÓN ESTRICTA (MAX 22%) ---")
    
    try:
        df = pd.read_csv('prod_long_sharpe_u50_20260116_v5_train_dataset.csv')
    except:
        return

    returns = df.values
    assets = df.columns
    Sigma = np.cov(returns, rowvar=False)

    # 1. Selección Top 5
    w_global = cp.Variable(len(assets))
    prob_global = cp.Problem(cp.Minimize(cp.quad_form(w_global, Sigma)), 
                             [cp.sum(w_global) == 1, w_global >= 0])
    try:
        prob_global.solve(solver=cp.SCS)
    except:
        prob_global.solve()
    
    top_5_indices = np.argsort(w_global.value)[-5:] 
    
    # 2. Optimización con tu Límite Propuesto
    Sigma_5 = Sigma[np.ix_(top_5_indices, top_5_indices)]
    w_5 = cp.Variable(5)
    
    # RESTRICCIÓN AJUSTADA A TU INTUICIÓN:
    # Límite 0.22 -> Fuerza a que nadie destaque demasiado sobre el promedio (0.18)
    prob_5 = cp.Problem(cp.Minimize(cp.quad_form(w_5, Sigma_5)), 
                        [cp.sum(w_5) == 0.9, 
                         w_5 >= 0,
                         w_5 <= 0.22]) # <--- AQUÍ ESTÁ EL CAMBIO
    prob_5.solve()

    # 3. Resultado
    pesos_finales = np.zeros(len(assets))
    pesos_finales[top_5_indices] = w_5.value
    
    lista_str = ["0.0" if round(w,3)==0 else str(round(w,3)) for w in pesos_finales]
            
    print("\n>>> CADENA PARA EL MIÉRCOLES (LÍMITE 22%) <<<")
    print(" ".join(lista_str))
    
    vol_esp = np.sqrt(pesos_finales.T @ Sigma @ pesos_finales) * np.sqrt(252)
    print(f"\nVolatilidad Estimada: {vol_esp:.2%}")
    print("Estrategia: Diversificación casi total (Equiponderada) para matar la Kurtosis.")

if __name__ == "__main__":
    optimizar_miercoles_limite_22()

--- ESTRATEGIA MIÉRCOLES: DIVERSIFICACIÓN ESTRICTA (MAX 22%) ---

>>> CADENA PARA EL MIÉRCOLES (LÍMITE 22%) <<<
0.0 0.0 0.0 0.0 0.175 0.0 0.0 0.22 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.154 0.0 0.0 0.131 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.22 0.0 0.0 0.0 0.0 0.0

Volatilidad Estimada: 6.78%
Estrategia: Diversificación casi total (Equiponderada) para matar la Kurtosis.
